# iPython Notebook for Segmenting New Scar Cases 

## Make sure to have the following python files in the same directory as this iPython notebook: 
1. training.py 
2. unet3d.py
3. cunet3d.py
4. unetplusplus3d.py
5. testing3D.py
6. scar_mean.npy

To use this notebook, ensure that your data meets the following requirements: 
1. Converted to a numpy array of in the formation (N_studies,256,256,16). You will need to pad your data to meet these requirements. The user can decide the best algorithm for padding, though note that the model was trained with zero-padding. For example, if you would like to segment 10 students, then your numpy array should have dimensions (10,256,256,16). 
2. Ensure that that the data type is unsigned integer 8 (uint8). 


In [6]:
# Required Libraries 
import numpy as np 
import os 

# Keras Libraries 
from tensorflow.keras.utils import to_categorical 
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf


# My libraries 
from unet3d import get_unet3D_multi 
from cunet3d import get_Cunet3D_mulit
from unetplusplus3d import get_unetpp3D_multi

from training import training_main 
from testing3D import testing_parallel

2023-12-07 15:43:08.669452: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [ ]:
# USER DATA INPUT

dpath = # Enter path with .npy array here 
shift = np.load(dpath+'scar_mean.npy')

imgs_ext = np.load(dpath) 
imgs_ext = (imgs_ext - scar_mean)/255 # Normalizes the data 
segs_ext = np.zeros(shape=imgs_ext.shape)



In [ ]:
# USER MODEL TYPE SELECTION AND MODEL DEFINITION 

model_type = # User can input the models. The selection is: U-Net, Cascaded U-Net, U-Net++. Please input the name of the model of choice here. 

if model_type == "U-Net": 
    path1 = "weights_unet.h5"
elif model_type == "Cascaded U-Net": 
    path1 = "weights_cunet.h5": 
elif model_type == "U-Net++": 
    path1 = "weights_unetpp.h5"
else: 
    print('Please enter one of the appropriate options: U-Net, Cascaded U-Net, or U-Net++')
    

# Image Parameters 
img_rows = 256
img_cols = 256
slices = 16 


# Model Parameters 
classes = 3 
layers = 4
min_convs = 4
kernel = (3,3,3)
lossfun = 'wcce_kld' # Need to write something for this later - might clean up model code as well 

# Hyperparameters 
decay = 0.98
learning_rate = 1e-3 
batches = 4
epochs_total = 100
epochs_batch = 5 
weights_init = (1,20,1000) 

strategy = tf.distribute.MirroredStrategy()

train_loss = np.zeros((epochs_total,))
train_dice = np.zeros((epochs_total,))
train_myo = np.zeros((epochs_total,))

val_loss = np.zeros((epochs_total,))
val_dice = np.zeros((epochs_total,))
val_myo = np.zeros((epochs_total,))


if model_type == "U-Net": 
    with strategy.scope():
        model = get_unet3D_multi(img_rows, img_cols, slices, classes, layers, min_convs, kernel, learning_rate, lossfun, weights_init)
    model.summary()
elif model_type == "Cascaded U-Net": 
    with strategy.scope():
        model = get_Cunet3D_mulit(img_rows, img_cols, slices, classes, layers, min_convs, kernel, learning_rate, lossfun, weights_init)
    model.summary()
    path1 = "weights_cunet.h5": 
elif model_type == "U-Net++": 
    with strategy.scope():
        model = get_unetpp3D_multi(img_rows, img_cols, slices, classes, layers, min_convs, kernel, learning_rate, lossfun, weights_init)
    model.summary()
else: 
    print('Please enter one of the appropriate options: U-Net, Cascaded U-Net, or U-Net++')



In [ ]:
# INPUT THE DIRECTORY WHERE YOU WOULD LIKE TO SAVE THE SEGMENTATION RESULTS 

spath = # Input directory for saving results here. 

if not os.path.exists(savepath):
    os.mkdir(savepath)

# Testing model on validation set only 
preds, contours = testing_parallel(savepath, model, imgs_ext, segs_ext, gpus =1, classes, shift)

In [ ]:
# PLOTTING REPRESENTATIVE SEGMENTATION MASKS 

m = # INPUT WHICH STUDY NUMBER YOU WOULD LIKE TO PLOT. IF YOU ONLY HAVE 1 STUDY, then m = 1 

for ii in range(16): 
    plt.subplot(4,4,ii+1)
    plt.imshow(preds[m,:,:,:])
    

In [ ]:
# PLOTTING REPRESENTATIVE SEGMENTATION MASKS 

m = # INPUT WHICH STUDY NUMBER YOU WOULD LIKE TO PLOT. IF YOU ONLY HAVE 1 STUDY, then m = 1 

for ii in range(16): 
    plt.subplot(4,4,ii+1)
    plt.imshow(contours[m,:,:,:,:])
    